# Dinâmica simbólica para UVMS (ROV + braço antropomórfico)

Notebook que monta as matrizes **M**, **C**, **H** e **G** para um UVMS de 12 DOF, usando uma matriz de excentricidades no mesmo formato do exemplo em MATLAB (ROV com 6 DOF de corpo rígido + braço de 6 DOF). O cálculo é 100% simbólico com SymPy.


In [ ]:
import sympy as sp
from robot_dynamics import uvms_dynamics

sp.init_printing(use_unicode=True)


## Excentricidades e ordem das juntas

Primeiro declaramos a ordem das juntas e a matriz de excentricidades (12x3). As três primeiras juntas são prismáticas do ROV (Dx, Dy, Dz), seguidas de três rotações do corpo (z, y, x) e das seis rotações do braço antropomórfico (z, y, y, z, y, z). Substitua os comprimentos simbólicos conforme necessário.


In [ ]:
# Juntas e variáveis generalizadas
q = sp.symbols('q1:13')
qd = sp.symbols('dq1:13')

axis_order = ['Dx','Dy','Dz','z','y','x','z','y','y','z','y','z']

# Comprimentos simbólicos para a cadeia do braço
l_p1, l1, l2, l3, l_tcp = sp.symbols('l_p1 l1 l2 l3 l_tcp')

excentricidades = sp.Matrix([
    [0, 0, 0],  # 1) Dx
    [0, 0, 0],  # 2) Dy
    [0, 0, 0],  # 3) Dz
    [0, 0, 0],  # 4) Rz
    [0, 0, 0],  # 5) Ry
    [0, 0, 0],  # 6) Rx
    [0, 0, l_p1],  # 7) pedestal -> junta 2
    [l1, 0, 0],    # 8) elo 1 -> elo 2
    [l2, 0, 0],    # 9) elo 2 -> elo 3
    [l3, 0, 0],    # 10) elo 3 -> punho
    [0, 0, 0],     # 11) junta esférica
    [0, 0, l_tcp], # 12) wrist center -> TCP
])

# Massas e inércias (valores genéricos/simbólicos)
masses = sp.symbols('m1:13')
inertias = [sp.diag(*sp.symbols(f'I{i+1}x I{i+1}y I{i+1}z')) for i in range(12)]

g = sp.symbols('g')
gravity_vec = sp.Matrix([0, 0, -g])


## Matrizes de dinâmica simbólica

Chamamos `uvms_dynamics` para obter as transformações homogêneas de cada junta (incluindo excentricidades), energia cinética/potencial e as matrizes **M**, **C**, **H** e **G** para todo o UVMS.


In [ ]:
transforms, T_energy, V_energy, M, C, H, G = uvms_dynamics(
    axis_order,
    excentricidades,
    q,
    qd,
    masses,
    inertias,
    gravity_vec,
)

M, C, H, G
